<a href="https://colab.research.google.com/github/aicuai/Book-SD-MasterGuide/blob/main/AICU_ComfyUI_LoRA_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComfyUIのインストール https://j.aicu.ai/ComfyLoRA

この Google Colab向け notebook は ComfyUIを初心者向けに簡単に起動できるように解説・設定しています。

原作は [ComfyUI-Manager](https://github.com/ltdrdata/ComfyUI-Manager) による [こちら](https://github.com/ltdrdata/ComfyUI-Manager/blob/main/notebooks/comfyui_colab_with_manager.ipynb)


AICU mediaによる解説記事は[ここ](https://note.com/aicu/n/n876fab5a0736)から読めます

■ ここからはじめる「ComfyUIマスターガイド」
https://note.com/aicu/n/n876fab5a0736


In [ ]:
# #@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
USE_CUSTOM_NODES = True #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['USE_CUSTOM_NODES'] = USE_CUSTOM_NODES
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

# #7025 The frontend will no longer be shipped in the main ComfyUI repo
# https://github.com/comfyanonymous/ComfyUI/issues/7025
!pip install -r requirements.txt

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f ".ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/nightly/windows_base_files/run_nvidia_gpu.bat" ] && chmod 755 .ci/nightly/windows_base_files/run_nvidia_gpu.bat
  ![ -f ".ci/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows/update.py" ] && chmod 755 .ci/update_windows/update.py
  ![ -f ".ci/update_windows/update_comfyui.bat" ] && chmod 755 .ci/update_windows/update_comfyui.bat
  ![ -f ".ci/update_windows/README_VERY_IMPORTANT.txt" ] && chmod 755 .ci/update_windows/README_VERY_IMPORTANT.txt
  ![ -f ".ci/update_windows/run_cpu.bat" ] && chmod 755 .ci/update_windows/run_cpu.bat
  ![ -f ".ci/update_windows/run_nvidia_gpu.bat" ] && chmod 755 .ci/update_windows/run_nvidia_gpu.bat

  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate
!pip3 install einops transformers>=4.25.1 safetensors>=0.3.0 aiohttp pyyaml Pillow scipy tqdm psutil
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde
!pip3 install kornia>=0.7.1 spandrel

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

  %cd $WORKSPACE

if OPTIONS['USE_CUSTOM_NODES']:
  %cd $WORKSPACE/custom_nodes

  !git clone https://github.com/jags111/efficiency-nodes-comfyui.git

  %cd $WORKSPACE

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies


### 各種モデルのダウンロード

In [ ]:
# Checkpoints

## SDXL
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P ./models/checkpoints/
!wget -c https://huggingface.co/SG161222/RealVisXL_V5.0_Lightning/resolve/main/RealVisXL_V5.0_Lightning_fp16.safetensors -P ./models/checkpoints/

# Loras
!wget -c "https://civitai.com/api/download/models/135867?type=Model&format=SafeTensor" -O ./models/loras/add-detail-xl.safetensors # Detail Tweaker XL
!wget -c https://huggingface.co/e-n-v-y/envy-zoom-slider-xl-01/resolve/main/EnvyZoomSliderXL01.safetensors -P ./models/loras/ # envy-zoom-slider-xl-01
!wget -c "https://civitai.com/api/download/models/277389?type=Model&format=SafeTensor" -O ./models/loras/ral-dissolve-sdxl.safetensors # Dissolve Style [LoRA 1.5+SDXL]

### ワークフローのダウンロード

In [ ]:
# Download workflows
!wget -c https://raw.githubusercontent.com/aicuai/Book-SD-MasterGuide/refs/heads/main/workflows/workflow_sdxl_lora.json -P ./user/default/workflows/
!wget -c https://raw.githubusercontent.com/aicuai/Book-SD-MasterGuide/refs/heads/main/workflows/workflow_detail-tweaker-xl.json -P ./user/default/workflows/
!wget -c https://raw.githubusercontent.com/aicuai/Book-SD-MasterGuide/refs/heads/main/workflows/workflow_detail-tweaker-xl_with_xy-plot.json -P ./user/default/workflows/
!wget -c https://raw.githubusercontent.com/aicuai/Book-SD-MasterGuide/refs/heads/main/workflows/workflow_envy-zoom-slider-xl.json -P ./user/default/workflows/
!wget -c https://raw.githubusercontent.com/aicuai/Book-SD-MasterGuide/refs/heads/main/workflows/workflow_envy-zoom-slider-xl_with_xy-plot.json -P ./user/default/workflows/

### ComfyUIの起動



In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server